<span style="color:orange; font-weight:bold">Note: To answer questions based on text documents, we recommend the procedure in <a href="https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb">Question Answering using Embeddings</a>. Some of the code below may rely on <a href="https://github.com/openai/openai-cookbook/tree/main/transition_guides_for_deprecated_API_endpoints">deprecated API endpoints</a>.</span>

In [1]:
import openai
import tiktoken
import os
import pandas as pd
import numpy as np
import time

Use tiktoken.get_encoding() to load an encoding by name.

The first time this runs, it will require an internet connection to download. Later runs won't need an internet connection.

In [2]:
encoding = tiktoken.get_encoding("cl100k_base")

Use tiktoken.encoding_for_model() to automatically load the correct encoding for a given model name.

In [3]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [4]:
with open("openai_token.txt", 'r') as fp:
    openai_token = fp.readline()
# Set your OpenAI API key here
openai.api_key = openai_token
# Set the environment variable
#os.environ["OPENAI_API_KEY"] = api_key

## Prompt preparation

In [5]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [6]:
def remove_non_ascii(sentence):
    return ''.join(char for char in sentence if ord(char) < 128)

In [7]:
with open('prompt/prompt_system_etd.txt', 'r') as fp:
    etd_system = fp.readlines()
    
with open('prompt/prompt_system_ps.txt', 'r') as fp:
    ps_system = fp.readlines()

In [8]:
" ".join(etd_system)

'Use the following step-by-step instructions to respond to user inputs.\n \n Step 1 - The user will provide you with text in triple quotes (Question: """insert text here"""). Find one sentence from the given text which can indicate the answer to "What is the user expecting to do or achieve?". The sentence only describing the system error or undesired behavior should not be considered. Use triple quotes to cite the found sentence. If the given text does not contain such sentence, write "No sentences found to indicate the user\'s expectation. The answer is NO." and skip Step 2 and Step 3.\n \n Step 2 - If found one sentence from Step 1, \n summarize the sentence into one or more patterns delimited by <pattern>..</pattern> according to the syntax and semantic of the sentence. The pattern could be expressed by some keywords and part-of-speech tags from the found sentence, which can explain why this sentence provides the answer to "What is the user expecting to do or achieve?".\n \n Step 3 

In [9]:
total_user_etd_len = num_tokens_from_string(" ".join(etd_system), "cl100k_base")
total_user_etd_len

254

In [10]:
" ".join(ps_system)

'Use the following step-by-step instructions to respond to user inputs.\n \n Step 1 - The user will provide you with text in triple quotes (Question: """insert text here"""). Find one sentence from the given text which can indicate the answer to "What is the error or undesired behavior of the system or code provided by the user?". The sentence only describing the user\'s expectation should not be considered. Use triple quotes to cite the found sentence. If the given text does not contain such sentence, write "No sentences found to indicate the error or undesired behavior. The answer is NO." and skip Step 2 and Step 3.\n \n Step 2 - If found one sentence from Step 1, \n summarize the sentence into one or more patterns delimited by <pattern>..</pattern> according to the syntax and semantic of the sentence. The pattern could be expressed by some keywords and part-of-speech tags from the found sentence, which can explain why this sentence provides the answer to "What is the error or undesi

In [11]:
total_user_ps_len = num_tokens_from_string(" ".join(ps_system), "cl100k_base")
total_user_ps_len

266

In [12]:
df_issue = pd.read_csv("prompt/issue.csv", encoding = "ISO-8859-1")
# df_issue = pd.read_csv("prompt/issue_wrong.csv")

# df_etd_example = pd.read_csv("experiment/random_pattern/ETD/etd_prompt_random_with_neg1.1.csv")
df_ps_example = pd.read_csv("experiment/random_pattern/PS/ps_prompt_random_with_neg1.1.csv")
#df_etd_example = pd.read_csv("experiment/frequent/ETD/etd_prompt_frequent1.csv")
#df_ps_example = pd.read_csv("experiment/frequent/PS/ps_prompt_frequent1.csv")

In [13]:
len(df_issue)

1000

In [14]:
issue_clean = [remove_non_ascii(issue) for issue in df_issue["issue"]]
df_issue["issue"] = issue_clean
df_issue.head()

,issue
0,"Hi, when I am trying to apply class dynamicall..."
1,is it possible to get a RouteConfig matched ag...
2,I have angular running from a .Net Core server...
3,"Hello everyone, I want to do sub menu with sea..."
4,I'm struggling with getting this logic [code]....


### for all examples

In [ ]:
# chat_issue = list(df_etd_example["issue"].values)
chat_issue = list(df_ps_example["issue"].values)
chat_issue

In [ ]:
# chat_output = list(df_etd_example["output"].values)
chat_output = list(df_ps_example["output"].values)
chat_output

### for random/frequent pattern

In [ ]:
df_etd_randexample = df_etd_example[df_etd_example["6_examples"]==1]

chat_issue = list(df_etd_randexample["issue"].values)
# chat_issue.extend(list(df_etd_randexample["issue_n"].values))
chat_issue

In [ ]:
chat_output = list(df_etd_randexample["output"].values)
# chat_output.extend(list(df_etd_randexample["output_n"].values))
chat_output

In [15]:
df_ps_randexample = df_ps_example[df_ps_example["2_examples"]==1]

chat_issue = list(df_ps_randexample["issue"].values)
# chat_issue.extend(list(df_ps_randexample["issue_n"].values))
chat_issue

['Question: """ any ideas on how i may be able to find the circular dependency? i tried adding the `--verbose` flag to the ng build command but that didn\'t really help tried using this tool [link], but it reports no circular dependencies. i\'ve also looked through the code and nothing really stands out """\nAnswer: ',
 'Question: """ Hi, is there any way to detect no router change? I mean situation when user click the same routerLink he is allready in i would like to implement some monkeyPatch for forcing angular 4 router to reinitialize route """\nAnswer: ']

In [16]:
chat_output = list(df_ps_randexample["output"].values)
# chat_output.extend(list(df_ps_randexample["output_n"].values))
chat_output

['The sentence """  but it reports no circular dependencies. """ uses <pattern>reports no JJ NNS</pattern> indicate that "reports no circular dependencies" is an undesired behavior. The answer is YES.',
 'The sentence """  is there any way to detect no router change? """ uses <pattern>detect no router change</pattern> but "no router change" is a noun phrase here to indicate the user\'s expectation instead of describing the error or undesired behavior, so the answer is NO.']

## Model running

In [17]:
for user_string,output_string in zip(chat_issue,chat_output):
    total_user_etd_len += num_tokens_from_string(user_string,"cl100k_base")
    total_user_etd_len += num_tokens_from_string(output_string,"cl100k_base")
    
print("ETD total input token:",total_user_etd_len)

ETD total input token: 976


In [17]:
for user_string,output_string in zip(chat_issue,chat_output):
    total_user_ps_len += num_tokens_from_string(user_string,"cl100k_base")
    total_user_ps_len += num_tokens_from_string(output_string,"cl100k_base")
    
print("PS total input token:",total_user_ps_len)

PS total input token: 487


In [18]:
ans1= []
start_i = 0
flag = True
while flag:
    try:
        for i in range(start_i,len(df_issue),1):
#         for i in range(start_i,start_i+7,1):
            issue = df_issue.loc[i]["issue"]
            input_str = "Question: \"\"\" {} \"\"\"\n Answer: ".format(issue)
    
            result = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
#             {"role": "system", "content": " ".join(etd_system)},
            {"role": "system", "content": " ".join(ps_system)},
            {"role": "user", "content": chat_issue[0]},
            {"role": "assistant", "content": chat_output[0]},
            {"role": "user", "content": chat_issue[1]},
            {"role": "assistant", "content": chat_output[1]},
#             {"role": "user", "content": chat_issue[2]},
#             {"role": "assistant", "content": chat_output[2]},
#             {"role": "user", "content": chat_issue[3]},
#             {"role": "assistant", "content": chat_output[3]},
#             {"role": "user", "content": chat_issue[4]},
#             {"role": "assistant", "content": chat_output[4]},
#             {"role": "user", "content": chat_issue[5]},
#             {"role": "assistant", "content": chat_output[5]},
#             {"role": "user", "content": chat_issue[6]},
#             {"role": "assistant", "content": chat_output[6]},
#             {"role": "user", "content": chat_issue[7]},
#             {"role": "assistant", "content": chat_output[7]},
#             {"role": "user", "content": chat_issue[8]},
#             {"role": "assistant", "content": chat_output[8]},
#             {"role": "user", "content": chat_issue[9]},
#             {"role": "assistant", "content": chat_output[9]},
#             {"role": "user", "content": chat_issue[10]},
#             {"role": "assistant", "content": chat_output[10]},
#             {"role": "user", "content": chat_issue[11]},
#             {"role": "assistant", "content": chat_output[11]},
#             {"role": "user", "content": chat_issue[12]},
#             {"role": "assistant", "content": chat_output[12]},
#             {"role": "user", "content": chat_issue[13]},
#             {"role": "assistant", "content": chat_output[13]},
#             {"role": "user", "content": chat_issue[14]},
#             {"role": "assistant", "content": chat_output[14]},
#             {"role": "user", "content": chat_issue[15]},
#             {"role": "assistant", "content": chat_output[15]},
#             {"role": "user", "content": chat_issue[16]},
#             {"role": "assistant", "content": chat_output[16]},
#             {"role": "user", "content": chat_issue[17]},
#             {"role": "assistant", "content": chat_output[17]},
#             {"role": "user", "content": chat_issue[18]},
#             {"role": "assistant", "content": chat_output[18]},
#             {"role": "user", "content": chat_issue[19]},
#             {"role": "assistant", "content": chat_output[19]},
#             {"role": "user", "content": chat_issue[20]},
#             {"role": "assistant", "content": chat_output[20]},
#             {"role": "user", "content": chat_issue[21]},
#             {"role": "assistant", "content": chat_output[21]},
#             {"role": "user", "content": chat_issue[22]},
#             {"role": "assistant", "content": chat_output[22]},
#             {"role": "user", "content": chat_issue[23]},
#             {"role": "assistant", "content": chat_output[23]},
#             {"role": "user", "content": chat_issue[24]},
#             {"role": "assistant", "content": chat_output[24]},
#             {"role": "user", "content": chat_issue[25]},
#             {"role": "assistant", "content": chat_output[25]},
#             {"role": "user", "content": chat_issue[26]},
#             {"role": "assistant", "content": chat_output[26]},
#             {"role": "user", "content": chat_issue[27]},
#             {"role": "assistant", "content": chat_output[27]},
#             {"role": "user", "content": chat_issue[28]},
#             {"role": "assistant", "content": chat_output[28]},
            {"role": "user", "content": input_str},
                ],
                max_tokens = 200,
                temperature = 0.8,
#                 seed = 30
        )
    
            ans1.append(result['choices'][0]['message']['content'])
        
        flag = False
    except openai.error.RateLimitError as e:
        print(f"RateLimitError: {e}")
        time.sleep(60)
        start_i = len(ans1)+0
    except Exception as e:
        print(f"An error occurred: {e}")
        break  # Exit the loop on other errors

In [ ]:
ans1= []

for i in range(len(df_issue)):
    issue = df_issue.loc[i]["issue"]
    input_str = "Question: \"\"\" {} \"\"\"\n Answer: ".format(issue)
    
    result = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": " ".join(etd_system)},
#             {"role": "system", "content": " ".join(ps_system)},
            {"role": "user", "content": chat_issue[0]},
            {"role": "assistant", "content": chat_output[0]},
            {"role": "user", "content": chat_issue[1]},
            {"role": "assistant", "content": chat_output[1]},
            {"role": "user", "content": chat_issue[2]},
            {"role": "assistant", "content": chat_output[2]},
            {"role": "user", "content": chat_issue[3]},
            {"role": "assistant", "content": chat_output[3]},
            {"role": "user", "content": chat_issue[4]},
            {"role": "assistant", "content": chat_output[4]},
            {"role": "user", "content": chat_issue[5]},
            {"role": "assistant", "content": chat_output[5]},
            {"role": "user", "content": chat_issue[6]},
            {"role": "assistant", "content": chat_output[6]},
            {"role": "user", "content": chat_issue[7]},
            {"role": "assistant", "content": chat_output[7]},
            {"role": "user", "content": chat_issue[8]},
            {"role": "assistant", "content": chat_output[8]},
            {"role": "user", "content": chat_issue[9]},
            {"role": "assistant", "content": chat_output[9]},
            {"role": "user", "content": chat_issue[10]},
            {"role": "assistant", "content": chat_output[10]},
            {"role": "user", "content": chat_issue[11]},
            {"role": "assistant", "content": chat_output[11]},
            {"role": "user", "content": chat_issue[12]},
            {"role": "assistant", "content": chat_output[12]},
            {"role": "user", "content": chat_issue[13]},
            {"role": "assistant", "content": chat_output[13]},
            {"role": "user", "content": chat_issue[14]},
            {"role": "assistant", "content": chat_output[14]},
            {"role": "user", "content": chat_issue[15]},
            {"role": "assistant", "content": chat_output[15]},
            {"role": "user", "content": chat_issue[16]},
            {"role": "assistant", "content": chat_output[16]},
            {"role": "user", "content": chat_issue[17]},
            {"role": "assistant", "content": chat_output[17]},
            {"role": "user", "content": chat_issue[18]},
            {"role": "assistant", "content": chat_output[18]},
            {"role": "user", "content": chat_issue[19]},
            {"role": "assistant", "content": chat_output[19]},
            {"role": "user", "content": chat_issue[20]},
            {"role": "assistant", "content": chat_output[20]},
            {"role": "user", "content": chat_issue[21]},
            {"role": "assistant", "content": chat_output[21]},
            {"role": "user", "content": chat_issue[22]},
            {"role": "assistant", "content": chat_output[22]},
            {"role": "user", "content": chat_issue[23]},
            {"role": "assistant", "content": chat_output[23]},
            {"role": "user", "content": chat_issue[24]},
            {"role": "assistant", "content": chat_output[24]},
#             {"role": "user", "content": chat_issue[25]},
#             {"role": "assistant", "content": chat_output[25]},
#             {"role": "user", "content": chat_issue[26]},
#             {"role": "assistant", "content": chat_output[26]},
#             {"role": "user", "content": chat_issue[27]},
#             {"role": "assistant", "content": chat_output[27]},
#             {"role": "user", "content": chat_issue[28]},
#             {"role": "assistant", "content": chat_output[28]},
            {"role": "user", "content": input_str},
            ],
        max_tokens = 100,
        temperature = 0
    )
    
    ans1.append(result['choices'][0]['message']['content'])

In [19]:
print(len(ans1))

1000


In [20]:
df = pd.DataFrame()
df["answer1"] = ans1

df.head(10)

,answer1
0,"The sentence """""" it is messing up my material ..."
1,No sentences found to indicate the error or un...
2,No sentences found to indicate the error or un...
3,No sentences found to indicate the error or un...
4,"The sentence """""" if API returns no data """""" u..."
5,No sentences found to indicate the error or un...
6,"The sentence """""" means that elements for both ..."
7,"The sentence """""" `Element implicitly has an 'a..."
8,No sentences found to indicate the error or un...
9,"The sentence """"""The only problem is with proje..."


In [21]:
df.to_csv("experiment/random_pattern/answer_rand.csv",index = None)

## Test Evaluation

In [22]:
from sklearn import metrics as mt

In [29]:
df_test = pd.read_csv("../Data/chat_pattern/chat_test_missing_1000.csv")
predictions_etd = df_test["y'_ETD"]
y_test = df_test["y_ETD"]
precison_etd = mt.precision_score(y_test, predictions_etd)
recall_etd = mt.recall_score(y_test, predictions_etd)
score_etd = mt.f1_score(y_test, predictions_etd)

print("precision:",round(precison_etd,3),"recall:",round(recall_etd,3),"F1:",round(score_etd,3))

precision: 0.506 recall: 0.852 F1: 0.635


In [30]:
predictions = df_test["y'_PS"]
y_test = df_test["y_PS"]
precison_ps = mt.precision_score(y_test, predictions)
recall_ps = mt.recall_score(y_test, predictions)
score_ps = mt.f1_score(y_test, predictions)

print("precision:",round(precison_ps,3),"recall:",round(recall_ps,3),"F1:",round(score_ps,3))

precision: 0.774 recall: 0.715 F1: 0.744


In [23]:
# ETD results

df_test_random = pd.read_csv("experiment/random_pattern/ETD/etd_random_withneg2.2_result(t=0.8).csv")
predictions_etd = df_test_random["y''_ETD"]
y_test = df_test_random["y_ETD"]
precison_etd = mt.precision_score(y_test, predictions_etd)
recall_etd = mt.recall_score(y_test, predictions_etd)
score_etd = mt.f1_score(y_test, predictions_etd)

print("precision:",round(precison_etd,3),"recall:",round(recall_etd,3),"F1:",round(score_etd,3))

precision: 0.854 recall: 0.344 F1: 0.49


In [28]:
# ETD results

df_test_random = pd.read_csv("experiment/random_pattern/ETD/result_missing/etd_rand1_5example_result2.csv")
predictions_etd = df_test_random["y''_ETD"]
y_test = df_test_random["y_ETD"]
precison_etd = mt.precision_score(y_test, predictions_etd)
recall_etd = mt.recall_score(y_test, predictions_etd)
score_etd = mt.f1_score(y_test, predictions_etd)

print("precision:",round(precison_etd,3),"recall:",round(recall_etd,3),"F1:",round(score_etd,3))

precision: 0.773 recall: 0.52 F1: 0.621


In [23]:
# PS results 

df_test_random = pd.read_csv("experiment/random_pattern/PS/ps_random_withneg12_result(t=0.8).csv")
predictions_ps = df_test_random["y''_PS"]
y_test = df_test_random["y_PS"]
precison_ps = mt.precision_score(y_test, predictions_ps)
recall_ps = mt.recall_score(y_test, predictions_ps)
score_ps = mt.f1_score(y_test, predictions_ps)

print("precision:",round(precison_ps,3),"recall:",round(recall_ps,3),"F1:",round(score_ps,3))

precision: 0.876 recall: 0.712 F1: 0.785
